In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots, Distributed

addprocs(4)

@everywhere begin
    
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots
    
nsp=10

samples=[collect(0:10:100); collect(100:50:1000)]

#Necessary functions

function alpha_gt(x::Float64, y::Float64, omega::Float64, t::Float64)

        al=exp(-((x-y)^2)/(omega^2))
        denom=(pi/2)*(erf(t/omega)-erf(-t/omega))
        al/=denom
    
        if(abs(x-y)>t)
            al=0
        end
    
    return al

end

#parameter combinations

reps=collect(1:20)
loci=[3,5,10,15,20]


pars=collect(Iterators.product(reps,loci))

function compsim(par::Tuple{Int64, Int64})
    
    rep=par[1]
    n=par[2]

    geno= collect(range(-1,stop=1,length=2*n+1))
    nt=length(geno)
    
    
    N=rand(Uniform(0,1.0), nsp,nt)
    Ng0= N ./ (sum.(eachrow(N)))
    Np0= 1000 .* Ng0 
    r=abs.(rand(Uniform(0,0.1),nsp))
    
    
    #Set up a probability matrix of an offspring having a phenotype x when the parents have the phenotypes v and w resp.
    # This code follows the diploid version of an exact hypergeometric model
    #from Shpak and Kondrashov (1999)
    haplR=zeros(Float64,n+1,n+1,n+1)

    for i in 0:n, j in 0:i, k in 0:min(n,(i+j))
        haplR[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),max(0, i+j-n):min(i, j)) .*
                              map(x->pdf(Binomial(i+j-2*x),k-x),collect(max(0, i+j-n):min(i,j))))
    end

    for k in 0:n
        haplR[:,:,1+k]=haplR[:,:,1+k]+transpose(haplR[:,:,1+k])
        for i1 in 0:n
            haplR[i1+1,i1+1,k+1] /= 2
        end
    end

    ind_haplR=zeros(Float64,2*n+1, 2*n+1)

    for k in 0:n
        for i in 0:n
            ind_haplR[1+i,1+k] = haplR[1+i,1,1+k]
            for j in 0:n
                ind_haplR[1+j+n,1+k]=haplR[1+n,1+j,1+k]
            end
        end
    end

    R=zeros(Float64,nt,nt,nt)

    for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
        R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .* 
                            ind_haplR[1+j,1+q .- (0:q)])
    end
    
    
    omegas=[0.1,0.5]
    ts=[0.2,0.5,2]
    a1s=[0.1,0.5,1]
    
    result=DataFrame()
        
    for i in omegas, j in ts, k in a1s
        
        omega=i
        t=j
        a1=k
        
        #Pre-calculate coefficients of competition between pairs of genotypes

        A1=zeros(Float64,nt,nt)
        A0=zeros(Float64,nt,nt)
        
        for i1 in 1:nt, i2 in 1:nt
    
            A1[i1,i2]=a1*alpha_gt(geno[i2],geno[i1],omega,t)

            A0[i1,i2]=alpha_gt(geno[i2],geno[i1],omega,t)
    
        end

        A1= A1 ./ 1000
        A0= A0 ./ 1000
        
        
        Ngen=deepcopy(Ng0)
        Np=deepcopy(Np0)
        
        #Start the simulation
        for m in 1:1000
    
            #Determine the extinct species
            Np[findall(sum(Np,dims=2) .< 10),:] .= 0

            if all(sum(Np,dims=2) ==0) 
                break
            else

                newgen=zeros(Float64,nsp,nt)

                #Reproduction event
                for i in 1:size(Ngen)[1]

                    probs=Ngen[i,:]*Ngen[i,:]'

                    for j in 1:size(R)[3]
                        newgen[i,j]=sum(probs.*R[:,:,j])
                    end
                end

                newp=newgen .* sum(Np,dims=2)

                #Selection event

                for i in 1:size(newp)[1], j in 1:size(newp)[2]

                    comps=sum(A1[j,:] .* newp[1:end .!=i,:]') + sum(A0[j,:] .* newp[i,:]) 
                    Np[i,j]=newp[i,j]+(newp[i,j]*(1-comps))

                end

                Np[findall(Np .<1)] .= 0
                Ngen= Np ./ sum(Np,dims=2)
            end

            if m ∈ samples

                dat1=DataFrame(nloci=n,reps=rep,omega=omega,t=t,a1=a1,time=m,sp=1:nsp,trmean=mean.(eachrow(geno' .*Ngen)))

                append!(result,dat1)
            end

        end 
        
    end
    
    CSV.write(string("compSK_",rep,"_",n,"_",".csv"),result)

end

end   
        


In [ ]:
pmap(compsim,pars)

In [ ]:
compsim